In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames: 
#        print(os.path.join(dirname, filename))  

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


train_data = pd.read_csv("/kaggle/input/coms4771-spring-2022-regression-competition/train_examples.csv")
train_label = pd.read_csv("/kaggle/input/coms4771-spring-2022-regression-competition/train_labels.csv")
test_data = pd.read_csv("/kaggle/input/coms4771-spring-2022-regression-competition/test_examples.csv")





 

In [2]:

train_data['duration'] = train_label['duration']
train_data = train_data[train_data['feature_9'] < 1000000]




In [3]:
y_train = train_data['duration']
x_train = train_data.iloc[:, 2:]
x_train = x_train.iloc[:, :-1]



In [4]:
test_data["feature_0"].astype("category")
x_test = test_data.iloc[:, 2:]


In [5]:
LassoCV_model = LassoCV()
LassoCV_model.fit(x_train, y_train)
optimal_alpha = LassoCV_model.alpha_
print(optimal_alpha)

32.279721173385866


In [6]:
lasso_model = Lasso(alpha = optimal_alpha)
lasso_model.fit(X=x_train, y=y_train)
lasso_predictions = lasso_model.predict(x_test)
print(lasso_predictions)

[1141.1233246   967.9494649  1053.68137183 ...  845.41825693 1659.67873038
 1717.39010632]


In [7]:
df_lasso = pd.DataFrame(lasso_predictions, columns = ['duration'])
df_lasso.insert(loc=0, column='id', value=np.arange(len(df_lasso)))
print(df_lasso)

          id     duration
0          0  1141.123325
1          1   967.949465
2          2  1053.681372
3          3   837.687423
4          4  1283.833222
...      ...          ...
99995  99995  1415.217565
99996  99996   856.117067
99997  99997   845.418257
99998  99998  1659.678730
99999  99999  1717.390106

[100000 rows x 2 columns]


In [8]:
df_lasso.to_csv("/kaggle/working/output6.csv", index=False)

In [9]:

print(lasso_model.score(x_train, y_train))

0.023843364159232938


In [10]:
params = { 'max_depth': [5],
           'learning_rate': [0.01],
           'n_estimators': [200],
           'colsample_bytree': [0.75, 0.8, 0.85]}
xgbr = xgb.XGBRegressor()
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1, cv=3, n_jobs = -1)
clf.fit(x_train, y_train)
print("Best parameters:", clf.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200}


In [11]:
xgb_model = xgb.XGBRegressor(max_depth=5, n_estimators=200, eval_metric ='mae', learning_rate=0.01, reg_alpha=30, subsample=0.8)
xgb_model.fit(x_train, y_train)
xgb_predictions = xgb_model.predict(x_test)
print(xgb_predictions)

[ 374.12796  583.4616   903.3415  ...  520.5629  1330.3694  1279.6189 ]


In [12]:
df_xgb = pd.DataFrame(xgb_predictions, columns = ['duration'])
df_xgb.insert(loc=0, column='id', value=np.arange(len(df_xgb)))
print(df_xgb)

          id     duration
0          0   374.127960
1          1   583.461609
2          2   903.341492
3          3   254.128265
4          4  1032.530273
...      ...          ...
99995  99995   369.044189
99996  99996   422.675232
99997  99997   520.562927
99998  99998  1330.369385
99999  99999  1279.618896

[100000 rows x 2 columns]


In [13]:
df_xgb.to_csv("/kaggle/working/output.csv", index=False)